# Training Prediction Models Directly Within PostgreSQL Using EvaDB
In this tutorial, we'll harness EvaDB's model training capabilities to predict home rental prices, showcasing how EvaDB seamlessly integrates AI into your PostgreSQL database.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/staging/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"/> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/staging/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"/> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/staging/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

## Setup

### Install and Launch the PostgreSQL Server

To kick things off, we'll start by setting up the PostgreSQL database backend. If you already have a PostgreSQL server up and running, you can skip this step and proceed directly to [installing EvaDB](#install-evadb).

In [1]:
!apt -qq install postgresql
!service postgresql start

The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 19 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package logrotate.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../00-logrotate_3.19.0-1ubuntu1.1_amd64.deb ...
Unpacking logrotate (3.19.0-1ubuntu1.1

### Create User and Database

In [8]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


### Prettify  Output

In [9]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
def pretty_print(df):
    return display(HTML( df.to_html().replace("\\n","<br>")))

### Installing EvaDB
<a id='install_evadb'></a>
We install EvaDB along with the necessary PostgreSQL and Ludwig dependencies.

In [10]:
%pip install --quiet "evadb[postgres,xgboost,ludwig]"

import evadb
cursor = evadb.connect().cursor()

## Load data into PostgresSQL


### Setting up a Data Source in EvaDB
To establish a direct connection between EvaDB and underlying database systems such as PostgreSQL, we will create a data source. This process entails supplying EvaDB with the connection credentials for the active PostgreSQL server.

In [11]:
params = {
    "user": "eva",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

,0
0,The database postgres_data has been successful...


### Loading Home Property Sales Data from CSV into PostgreSQL

In this step, we will import the [House Property Sales](https://www.kaggle.com/datasets/htagholdings/property-sales?resource=download) dataset into our PostgreSQL database. If you already have the data stored in PostgreSQL and are ready to proceed with the prediction model training, feel free to skip this section and head directly to the [model training process](#train-the-prediction-model).


In [12]:
!mkdir -p content
!wget -nc -O /content/home_rentals.csv https://www.dropbox.com/scl/fi/gy2682i66a8l2tqsowm5x/home_rentals.csv?rlkey=e080k02rv5205h4ullfjdr8lw&raw=1

--2023-10-26 18:33:10--  https://www.dropbox.com/scl/fi/gy2682i66a8l2tqsowm5x/home_rentals.csv?rlkey=e080k02rv5205h4ullfjdr8lw
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8d990fb5d1437b8351f26de88a.dl.dropboxusercontent.com/cd/0/inline/CGVkpQp8MQys_mL2iZ4WZL0mLTIryv1kHm9c6YlkGomsP279V5incJvOz05jfx5oqY0zKLKoMNcksLTrXBuk3MgOYSnfeHIjuKAzyWM5fZKfHS1qfV9_8EQsi--9jZx53LFu_MvgxiKWMys7t4DrWF_1/file# [following]
--2023-10-26 18:33:11--  https://uc8d990fb5d1437b8351f26de88a.dl.dropboxusercontent.com/cd/0/inline/CGVkpQp8MQys_mL2iZ4WZL0mLTIryv1kHm9c6YlkGomsP279V5incJvOz05jfx5oqY0zKLKoMNcksLTrXBuk3MgOYSnfeHIjuKAzyWM5fZKfHS1qfV9_8EQsi--9jZx53LFu_MvgxiKWMys7t4DrWF_1/file
Resolving uc8d990fb5d1437b8351f26de88a.dl.dropboxusercontent.com (uc8d990fb5d1437b8351f26de88a.dl.dropboxusercontent.com)... 162.125.3.15, 26

In [13]:
cursor.query("""
  USE postgres_data {
    CREATE TABLE IF NOT EXISTS home_rentals (
      number_of_rooms INT,
      number_of_bathrooms INT,
      sqft INT,
      location VARCHAR(128),
      days_on_market INT,
      initial_price INT,
      neighborhood VARCHAR(128),
      rental_price FLOAT
    )
  }
""").df()

,status
0,success


In [14]:
cursor.query("""
  USE postgres_data {
    COPY home_rentals(number_of_rooms, number_of_bathrooms, sqft, location, days_on_market, initial_price, neighborhood, rental_price)
    FROM '/content/home_rentals.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

,status
0,success


### Preview the Data


Within the home_rentals table, there are 8 columns at our disposal. Our objective is to utilize the remaining 7 columns to make predictions for the rental_price.

In [17]:
cursor.query("SELECT * FROM postgres_data.home_rentals LIMIT 3;").df()

,rental_price,number_of_bathrooms,sqft,initial_price,number_of_rooms,days_on_market,location,neighborhood
0,2167.0,1,674,2167,1,1,good,downtown
1,1883.0,1,554,1883,1,19,poor,westbrae
2,2431.0,1,529,2431,0,3,great,south_side


## Training Model

Next, we employ EvaDB to facilitate the training of an ML model, which will enable us to predict `home rental prices`.

### Train the prediction Model using Ludwig
For this purpose, we harness the capabilities of the [ludwig](https://ludwig.ai/latest/) engine to train our prediction model. We employ the `automl` feature to automatically determine the optimal hyperparameters. Keep in mind that `TIME_LIMIT` specifies the time budget allocated for the training process.

In [ ]:
cursor.query("""
  CREATE OR REPLACE FUNCTION PredictHouseRent FROM
  ( SELECT * FROM postgres_data.home_rentals )
  TYPE Ludwig
  PREDICT 'rental_price'
  TIME_LIMIT 3600;
""").df()

### Train the prediction Model using XGBoost
With XGBoost integration, we harness the capabilities of the [XGBoost](https://xgboost.readthedocs.io/en/stable/) engine to train our prediction model. We employ the `Flaml` AutoML model to automatically determine the optimal hyperparameters. We can utilize `TIME_LIMIT` to specify the time budget allocated for the training process. Similarly you can use the `METRIC` parameter to determine the accuracy/error metric for training the model. `TASK` parameter can be used to specify whether you want to perform a `classification` task or `regression` task.

In [19]:
cursor.query("""
  CREATE OR REPLACE FUNCTION PredictHouseRent FROM
  ( SELECT * FROM postgres_data.home_rentals )
  TYPE XGBoost
  PREDICT 'rental_price'
  TIME_LIMIT 180
  METRIC 'rmse'
  TASK 'regression';
""").df()

[flaml.automl.logger: 10-26 18:42:28] {1679} INFO - task = regression
[flaml.automl.logger: 10-26 18:42:28] {1690} INFO - Evaluation method: cv
[flaml.automl.logger: 10-26 18:42:28] {1788} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 10-26 18:42:28] {1900} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl.logger: 10-26 18:42:28] {2218} INFO - iteration 0, current learner xgboost
[flaml.automl.logger: 10-26 18:42:28] {2344} INFO - Estimated sufficient time budget=1240s. Estimated necessary time budget=1s.
[flaml.automl.logger: 10-26 18:42:28] {2391} INFO -  at 0.2s,	estimator xgboost's best error=873.7432,	best estimator xgboost's best error=873.7432
[flaml.automl.logger: 10-26 18:42:28] {2218} INFO - iteration 1, current learner xgboost
[flaml.automl.logger: 10-26 18:42:28] {2391} INFO -  at 0.3s,	estimator xgboost's best error=873.7432,	best estimator xgboost's best error=873.7432
[flaml.automl.logger: 10-26 18:42:28] {2218} INFO - iteration 2, current 

,0
0,Function PredictHouseRent added to the database.


### Utilizing the Prediction Model
Following the model training, we proceed to employ the `PredictHouseRent`` model to make predictions for home rental prices.

In [20]:
cursor.query("SELECT PredictHouseRent(*) FROM postgres_data.home_rentals LIMIT 10;").df()

,rental_price
0,2166.502930
1,1881.248047
2,2429.649170
3,5504.289062
4,2272.817139
5,4125.753906
6,2222.618164
7,2100.471191
8,3870.793945
9,2043.048096


We have the option to utilize a `LATERAL JOIN` to compare the actual rental prices in the `home_rentals` dataset with the predicted rental prices generated by the trained model, `PredictHouseRent`.

In [21]:
cursor.query("""
  SELECT rental_price, predicted_rental_price FROM postgres_data.home_rentals
  JOIN LATERAL PredictHouseRent(*) AS Predicted(predicted_rental_price) LIMIT 10;
""").df()

,rental_price,predicted_rental_price
0,2167.000,2167.564697
1,1883.000,1881.478516
2,2431.000,2430.222656
3,5510.000,5506.156738
4,2272.000,2274.904297
5,4123.812,4123.654785
6,2224.000,2224.599609
7,2104.000,2101.194824
8,3861.000,3866.042725
9,2041.000,2043.027222
